In [ ]:
import pandas as pd
import os

In [ ]:
# SETTINGS
path = '../010-add_coordinates/coordinates/'
filter_df = pd.read_csv('categories.csv', sep = ';', encoding = "ISO-8859-1")
filter_df

In [ ]:
filter_df = filter_df.loc[~filter_df.SELECT.isna(), ['SELECT', 'Geometry Code', 'Layer', 'fclass']]
#filter_df['Geometry Code'] = filter_df['Geometry Code'].str.pad(4, side = 'right', fillchar = '.')
filter_df['Geometry Code'] = filter_df['Geometry Code'].str.strip('0')

In [ ]:
granularity = 4 - filter_df['Geometry Code'].str.len()
multiplier = 10 ** granularity

In [ ]:
filter_df['Geometry Code'] = filter_df['Geometry Code'].astype(int)

filter_df['low_threshold'] = multiplier * filter_df['Geometry Code']
filter_df['high_threshold'] = multiplier * filter_df['Geometry Code'] + (multiplier - 1)

filter_df['fclass'] = filter_df['fclass'].fillna(filter_df['Layer'])
filter_df['granularity'] = granularity

filter_df = filter_df.sort_values(by=['granularity'])

filter_df

In [ ]:
file_list = os.listdir(path)

filter_regex = '|'.join(r"{}".format(x) for x in filter_df['Geometry Code'])

In [ ]:
filter_regex

In [ ]:
df_dict = {}

for file_name in file_list:

    df_dict[file_name] = pd.read_csv(path + file_name)

    df_dict[file_name].drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
filter_df_to_dict = filter_df[['fclass', 'low_threshold', 'high_threshold']]
filter_df_to_dict.drop_duplicates(subset = ['fclass'], inplace = True)
filter_df_to_dict

In [ ]:
filter_df_to_dict.index = filter_df_to_dict.fclass
filter_df_to_dict.drop('fclass', axis = 1, inplace = True)
filter_dict = filter_df_to_dict.to_dict('index')

In [ ]:
filter_df

In [ ]:
# df = df_dict['coordinates_gis_osm_pois_free_1.csv']
#
# df['fclass'] = 0
#
# for fclass, value_dict in filter_dict.items():
#
#     df.loc[(df.code >= value_dict['low_threshold']) & (df.code <= value_dict['high_threshold']),'fclass'] = fclass
#
#     final_df = df.merge(filter_df[['fclass', 'SELECT']], on = 'fclass', how = 'left')
#
#     final_df = final_df[final_df.fclass != 0]
#
# final_df

In [ ]:
filtered_df_dict = {}

for df_name, df in df_dict.items():

    print(df_name)

    df['fclass'] = 0

    for fclass, value_dict in filter_dict.items():

        df.loc[(df.code >= value_dict['low_threshold']) & (df.code <= value_dict['high_threshold']),'fclass'] = fclass

        final_df = df.merge(filter_df[['fclass', 'SELECT']], on = 'fclass', how = 'left')

        final_df = final_df.drop_duplicates()

        final_df = final_df[final_df.fclass != 0]

    filtered_df_dict[df_name] = final_df

In [ ]:
for df_name in filtered_df_dict.keys():
    print('DATAFRAME NAME: ', df_name)
    print('ORIGINAL DATAFRAME SIZE: ', df_dict[df_name].shape)
    print('FILTERED DATAFRAME SIZE: ', filtered_df_dict[df_name].shape)
    print('DIFFERENCE: ', df_dict[df_name].shape[0] - filtered_df_dict[df_name].shape[0])
    print(' ')

In [ ]:
completed_df_dict = {}

for df_name, df in filtered_df_dict.items():

    if df.shape[0] > 0:
        # completed_df_dict[df_name] = df[~df.name.isna()]
        completed_df_dict[df_name] = df

In [ ]:
for df_name in completed_df_dict.keys():
    print('DATAFRAME NAME: ', df_name)
    print('ORIGINAL DATAFRAME SIZE: ', df_dict[df_name].shape)
    print('FILTERED DATAFRAME SIZE: ', filtered_df_dict[df_name].shape)
    print('COMPLETE DATAFRAME SIZE: ', completed_df_dict[df_name].shape)
    print('DIFFERENCE: ', df_dict[df_name].shape[0] - completed_df_dict[df_name].shape[0])
    print(' ')

In [ ]:
os.makedirs('filtered/', exist_ok=True)

for df_name, df in completed_df_dict.items():

    file_name = 'filtered_' + df_name[:-4] + '.csv'
    df.to_csv('filtered/' + file_name)